# Chapter 11 가중치 그래프

## 11.2 가중치 그래프의 표현
* 인접 행렬을 통한
* 인접 리스트를 통한

### 인접 행렬: 간단한 연산

In [7]:
vertex =   ['A',    'B',    'C',    'D',    'E',    'F',    'G' ]
weight = [ [None,	29,		None,	None,	None,   10,		None],
           [29,	None,	16,		None,	None,	None,	15  ],
           [None,	16,		None,	12,		None,	None,	None],
           [None,	None,   12,		None,	22,		None,	18  ],
           [None,	None,	None,   22,		None,	27,		25  ],
           [10,	None,	None,	None,   27,		None,	None],
           [None,  15,		None,   18,		25,		None,	None]]    
graph = (vertex, weight)	

In [9]:
# 가중치의 합
def weightSum(vlist, W):
    sum = 0
    for i in range(len(vlist)):
        for j in range(i+1, len(vlist)):
            if W[i][j] != None:
                sum += W[i][j]
    return sum

In [10]:
print('AM : weight sum = ', weightSum(vertex, weight))

AM : weight sum =  174


In [15]:
# 모든 간선 출력
def printAllEdges(vlist, W):
    for i in range(len(vlist)):
        for j in range(i+1, len(W[i])):  # 표기에 있어서 위 코드와 통일성이 없는듯?
            if W[i][j] != None and W[i][j] != 0:
                print(f"({vlist[i]}, {vlist[j]}, {W[i][j]})", end=' ')
    print()

In [16]:
printAllEdges(vertex, weight)

(A, B, 29) (A, F, 10) (B, C, 16) (B, G, 15) (C, D, 12) (D, E, 22) (D, G, 18) (E, F, 27) (E, G, 25) 


### 인접 리스트: 간단한 연산

In [18]:
graphAL ={'A' : set([('B',29),('F',10)        ]),
        'B' : set([('A',29),('C',16), ('G',15)]),
        'C' : set([('B',16),('D',12)          ]),
        'D' : set([('C',12),('E',22), ('G',18)]),
        'E' : set([('D',22),('F',27), ('G',25)]),
        'F' : set([('A',10),('E',27)          ]),
        'G' : set([('B',15),('D',18), ('E',25)]) }

In [19]:
# 가중치의 합
def weightSum(graph):
    sum = 0
    for v in graph:
        for e in graph[v]:
            sum += e[1]
    return sum//2

In [20]:
# Test
weightSum(graphAL)

174

In [21]:
# 모든 간선 출력
def printAllEdges(graph):
    for v in graph:
        for e in graph[v]:
            print(f"({v}, {e[0]}, {e[1]})", end=' ')

In [24]:
# Test
# 간선은 이중 출력됨
printAllEdges(graphAL)

(A, F, 10) (A, B, 29) (B, G, 15) (B, C, 16) (B, A, 29) (C, B, 16) (C, D, 12) (D, G, 18) (D, E, 22) (D, C, 12) (E, F, 27) (E, D, 22) (E, G, 25) (F, E, 27) (F, A, 10) (G, B, 15) (G, D, 18) (G, E, 25) 

## 11.3 최소비용 신장트리

### 최소비용 신장트리란?
    * 간선들의 가중치 합이 최소인 신장 트리
     - 반드시 (n-1)개의 간선만 사용
     - 사이클이 포함되면 안됨

    * MST의 응용
     - 도로, 통신, 배관등의 건설 시 모두 연결하면서 길이/비용을 최소화
     - 전기 회로에서 단자를 모두 연결하면서 전선의 길이를 최소화

### Kruskal 알고리즘
    kruskal()
    1. 그래프의 모든 간선을 가중치에 따라 오름차순으로 정렬한다.
    2. 가장 가중치가 작은 간선 e를 뽑는다.
    3. e를 신장트리에 넣었을 때 사이클이 생기면 넣지 않고 2번으로 이동한다.
    4. 사이클이 생기지 않으면 최소 신장 트리에 삽입한다.
    5. n-1개의 간선이 삽입될 때 까지 2번으로 이동한다.

### union-find 알고리즘
* 간선 추가 시 사이클 검사 과정에 쓰임
* union : 두 집합의 합집합을 만드는 연산
* find  : 원소가 속한 집합을 찾는 연산

union-find 구현

In [4]:
parent = []  # 각 노드의 부모노드 인덱스
set_size = 0 # 전체 집합의 개수

def init_set(nSets): # 집합의 초기화 함수
    global set_size, parent
    set_size = nSets;
    for i in range(nSets):
        parent.append(-1)  # 각각이 고유의 집합(부모가 -1)

def find(id): # 정점 id가 속한 집합의 대표번호 탐색
    while parent[id] >= 0:
        id = parent[id]
    return id

def union(s1, s2): # 두 집합을 병합(s1을 s2에 병합시킴)
    global set_size
    parent[s1] = s2
    set_size = set_size - 1 # 집합의 개수가 줄어듬

Kruskal의 MST 알고리즘

In [5]:
def MSTKruskal(vertex, adj):    # 매개변수: 정점리스트, 인접행렬
    vsize = len(vertex)         # 정점의 개수
    init_set(vsize)             # 정점 집합 초기화
    eList = []                  # 간선 리스트

    for i in range(vsize):      # 모든 간선을 리스트에 넣음
        for j in range(i+1, vsize):
            if adj[i][j] != None:
                eList.append((i, j, adj[i][j])) # 튜플로 저장

    # 간선 리스트를 가중치의 내림차순으로 정렬: 람다 함수 사용
    eList.sort(key = lambda e : e[2], reverse=True)

    edgeAccepted = 0
    while edgeAccepted < vsize -1:
        e = eList.pop(-1)
        uset = find(e[0])
        vset = find(e[1])

        if uset != vset:
            print(f"간선 추가 : ({vertex[e[0]]}, {vertex[e[1]]}, {e[2]})")
            union(uset, vset)
            edgeAccepted += 1

테스트

In [6]:
vertex =   ['A',    'B',    'C',    'D',    'E',    'F',    'G' ]
weight = [ [None,	29,		None,	None,	None,   10,		None],
           [29,	None,	16,		None,	None,	None,	15  ],
           [None,	16,		None,	12,		None,	None,	None],
           [None,	None,   12,		None,	22,		None,	18  ],
           [None,	None,	None,   22,		None,	27,		25  ],
           [10,	None,	None,	None,   27,		None,	None],
           [None,  15,		None,   18,		25,		None,	None]]    

print("MST By Kruskal's Algorithm")
MSTKruskal(vertex, weight)

MST By Kruskal's Algorithm
간선 추가 : (A, F, 10)
간선 추가 : (C, D, 12)
간선 추가 : (B, G, 15)
간선 추가 : (B, C, 16)
간선 추가 : (D, E, 22)
간선 추가 : (E, F, 27)
